## <b><font color='darkblue'>LangChain Preface</font></b>
([article source](https://dev.to/timesurgelabs/how-to-use-langchain-in-10-minutes-56e2)) <b><font size='3ptx'>[LangChain](https://www.langchain.com/) is a powerful library for Python and Javascript/Typescript that allows you to quickly prototype large language model applications.</font></b>

It allows you to chain together LLM tasks (<font color='brown'>hence the name</font>) and even allows you to run autonomous agents quickly and easily. Today we will be going over the basics of chains, so you can hit the ground running with your newest LLM projects!

### <b><font color='darkgreen'>Getting Started</font></b>
There are a few basic concepts you’ll need to understand in order to get started. <b>Chains</b> can be thought of as a list of actions to take with an LLM or multiple LLM calls in the list. A chain is made up of 3 simple parts.
* [**Prompt Template**](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)- so you can quickly change inputs without changing the prompt.
* [**LLM**](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) - The AI that actually runs your prompts.
* [**Output Parsers**](https://python.langchain.com/docs/modules/model_io/output_parsers/) - Converts the output into something useful, usually just another string.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence
from langchain.chains.sequential import SimpleSequentialChain
from dotenv import load_dotenv

# loads the .env file
load_dotenv()

### <b><font color='darkgreen'>Writing the Chain</font></b>
<b><font size='3ptx'>For this example, we’re going to write a chain that generates a TikTok script for an educational channel</font></b>. First, we need to generate a description for the TikTok. We will use prompt templating so we can reuse the prompt later.

In [ ]:
description_prompt = PromptTemplate.from_template(
    "Write me a description for a TikTok about {topic}")

This can then be used in a chain. Before we can define a chain, we need to define an LLM for the chain to use. LangChain recommends that most users should use The [**ChatOpenAI**](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) class to get the cost benefits and simplicity of the ChatGPT API.

Here is what your chain should look like now:

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# description_chain = LLMChain(llm=llm, prompt=description_prompt, verbose=True)
llm_chain = RunnableSequence(first=description_prompt, last=llm)

In [ ]:
output = llm_chain.invoke({'topic': 'Cats are cool'})

In [ ]:
print(output.content)

Now that we have a description, we need to have it write a script. This is where chaining comes in - we can sequentially call the LLM again using a slightly different prompt. First, let’s define a new prompt for the next chain:

In [ ]:
script_prompt = PromptTemplate.from_template(
    "Write me a script for a TikTok given the following description: {description}")

In [ ]:
script_llm_chain = RunnableSequence(first=script_prompt, last=llm)
script_llm_chain

In [ ]:
script_output = script_llm_chain.invoke({'description': output.content})

In [ ]:
print(script_output.content)

<b>Using them like this is fine, but what if we want to chain them together?</b> That’s where Sequential Chains comes in. These allow you to tie multiple chains into a single function call, with them executing in order they are defined. There are two types of sequential chains, we’re just going to focus on the simple sequential chain ([**SimpleSequentialChain**](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html)). Edit the Chain import line to the following:
```python
from langchain.chains.sequential import SimpleSequentialChain
```

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

description_chain = LLMChain(llm=llm, prompt=description_prompt)
script_chain = LLMChain(llm=llm, prompt=script_prompt)

tiktok_chain = SimpleSequentialChain(chains=[description_chain, script_chain], verbose=True)

In [ ]:
script = tiktok_chain.invoke("cats are cool")

In [ ]:
print(script)

### <b><font color='darkgreen'>Summary</font></b>
<b><font size='3ptx'>[LangChain](https://python.langchain.com/) is a game-changer for anyone looking to quickly prototype large language model applications</font></b>. In just a few minutes, we've walked through the process of creating chains, defining prompts, and even chaining together multiple LLM calls to create a dynamic TikTok script.

The power of LangChain lies in its simplicity and flexibility. Whether you're a seasoned developer or just starting out, LangChain's intuitive design allows you to harness the capabilities of large language models like never before. From generating creative content to running autonomous agents, the possibilities are endless.

## <b><font color='darkblue'>How I Made an AI Agent in 10 Minutes with LangChain</font></b>
([article source](https://dev.to/timesurgelabs/how-to-make-an-ai-agent-in-10-minutes-with-langchain-3i2n)) <b><font size='3ptx'>[LangChain](https://www.langchain.com/)is a powerful library that allows you to quickly prototype large language model applications. It allows you to chain together LLM tasks (hence the name) and even allows you to run autonomous agents quickly and easily.</b></font>

From this section, we'll explore how to create agents and define custom tools that those agents can use.

### <b><font color='darkgreen'>Quick Concepts</font></b>
[**Agents**](https://python.langchain.com/docs/modules/agents) are a way to run an LLM in a loop in order to complete a task. Agents are defined with the following:
* [**Agent Type**](https://python.langchain.com/docs/modules/agents/agent_types/) - This defines how the Agent acts and reacts to certain events and inputs. For this tutorial we will focus on the [ReAct Agent Type](https://python.langchain.com/docs/modules/agents/agent_types/react.html).
* [**LLM**](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) - The AI that actually runs your prompts.
* [**Tools**](https://python.langchain.com/docs/modules/agents/tools/) - These are Python (or JS/TS) functions that your Agent can call to interact with the world outside of itself. These can be as simple or as complex as you want them to be! Many tools make a [Toolkit](https://python.langchain.com/docs/modules/agents/toolkits/). There are [many toolkits already available](https://python.langchain.com/docs/integrations/toolkits/) built-in to LangChain, but for this example we’ll make our own.

### <b><font color='darkgreen'>Agents</font></b>
<b><font size='3ptx'>Agents use a combination of an LLM (or an LLM Chain) as well as a Toolkit in order to perform a predefined series of steps to accomplish a goal.</font></b> For this example, we’ll create a couple of custom tools as well as LangChain’s provided DuckDuckGo search tool to create a research agent

#### <b>Importing Necessary Libraries</b>

In [ ]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

# loads the .env file
load_dotenv()

Here's a breakdown of the imports:
* [**requests**](https://pypi.org/project/requests/): A popular Python library for making HTTP requests.
* [**BeautifulSoup**](https://pypi.org/project/beautifulsoup4/): A library for web scraping purposes to pull the data out of HTML and XML files.
* [**load_dotenv**](https://pypi.org/project/python-dotenv/): A method to load environment variables from a <font color='olive'>.env</font> file.
* **LangChain specific imports**: These are specific to the LangChain framework and are used to define tools, prompts, chat models, chains, and agents.

#### <b>Setting Up the DuckDuckGo Search Tool</b>
This initializes the DuckDuckGo search tool provided by [**LangChain**](https://www.langchain.com/). It allows you to search the web using DuckDuckGo and retrieve the results.

In [ ]:
ddg_search = DuckDuckGoSearchResults()

#### <b>Defining Headers for Web Requests</b>
This sets a user-agent header for our web requests. Some websites might block requests that don't have a user-agent set, thinking they're from bots.

In [ ]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

#### <b>Parsing HTML Content</b>
This function takes in HTML content, uses BeautifulSoup to parse it, and then extracts all the text from it.

In [ ]:
def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

#### <b>Fetching Web Page Content</b>
This function fetches the content of a web page using the [**requests**](https://pypi.org/project/requests/) library and then parses the HTML to extract the text.

In [ ]:
def fetch_web_page(url: str) -> str:
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

#### <b>Creating the Web Fetcher Tool</b>
Here, we're creating a new tool using the [**Tool**.from_function](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.Tool.html#langchain_core.tools.Tool.from_function) method. This tool will use our `fetch_web_page function` to fetch and parse web pages:

In [ ]:
web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Fetches the content of a web page"
)

#### <b>Setting Up the Summarizer</b>
This section sets up a summarizer using the ChatOpenAI model from LangChain. We define a prompt template for summarization, create a chain using the model and the prompt, and then define a tool for summarization. We use ChatGPT 3, 5 16k context as most web pages will exceed the 4k context of ChatGPT 3.5.

In [ ]:
prompt_template = "Summarize the following content: {content}"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func=llm_chain.run,
    name="Summarizer",
    description="Summarizes a web page"
)

#### <b>Initializing the Agent</b>
Here, we're initializing an agent with the tools we've defined. This agent will be able to search the web, fetch web pages, and summarize them. Notice how we can re-use the LLM from the summarize tool.

In [ ]:
tools = [ddg_search, web_fetch_tool, summarize_tool]

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
)

### <b><font color='darkgreen'>Running the Agent</font></b>
Finally, we define a prompt for our agent and run it. The agent will search the web for information about the Python’s Requests Library, fetch the content fetch some of the content, and then summarize it.

In [ ]:
prompt = (
    "Research how to use the requests library in Python."
    "Use your tools to search and summarize content into a guide on how to use the requests library.")

In [ ]:
output = agent.run(prompt)

### <b><font color='darkgreen'>Experimentation</font></b>
<b><font size='3ptx'>In this section, we'll explore how to modify the code from the blog post draft to use the experimental Plan and Execute agent</font></b>. The Plan and Execute agent accomplishes an objective by first planning what to do, then executing the sub-tasks. The planning is almost always done by an LLM, while the execution is usually done by a separate agent equipped with tools.

First, install the [**LangChain experimental package**](https://pypi.org/project/langchain-experimental/).
> pip install langchain_experimental

Then you can import the necessary modules from the <b><font color='blue'>langchain_experimental.plan_and_execute</font></b> package:

In [ ]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

Load the planner and executor:

In [ ]:
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)

Initialize the Plan and Execute agent:

In [ ]:
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True, handle_parsing_errors=True)

Run the agent with a prompt:

In [ ]:
prompt = '''Research how to use the print "Hello word" in Python. Use your tools to search and summarize content into a guide on how to achieve the goal.'''

In [ ]:
result = agent.run(prompt)

In [ ]:
print(result)

<font size='3ptx'><b>In this example, the agent will first plan the steps needed to accomplish the objective, then execute the sub-tasks using the tools provided</b></font>. The agent will search the web for information about the requests library in Python, fetch the content of the relevant results, and then summarize them into a guide.

Note that the Plan and Execute agent is experimental and may not work as expected in all cases. However, it can be a powerful tool for automating complex tasks that require multiple steps and interactions with external tools.

### <b><font color='darkgreen'>Summary</font></b>
<b><font size='3ptx'>LangChain is a game-changer for anyone looking to quickly prototype large language model applications. In just a few minutes, we’ve walked through the process of creating agents, defining custom tools, and even experimenting with the experimental Plan and Execute agent to automate complex tasks.</font></b>

The power of LangChain lies in its simplicity and flexibility. Whether you’re a seasoned developer or just starting out, LangChain’s intuitive design allows you to harness the capabilities of large language models like never before. From generating creative content to running autonomous agents, the possibilities are endless.

## <b><font color='darkblue'>How I Use Google's Gemini Pro with LangChain</font></b>
<b><font size='3ptx'>Google's Gemini Pro is one of the newest LLMs publicly available, and to the surprise of some its relatively price competitive.</font></b>

Its effectively free while you're developing, and after you development is complete its relatively cheap, costing around `$0.00025 per 1K` characters (characters, not tokens like OpenAI), which is slightly more expensive than GPT-3.5-Turbo, and `$0.0025 per image`, which is effectively the same as OpenAI's GPT-4).

### <b><font color='darkgreen'>Okay, I get it, how do I use it?</font></b>
First of all, we need to install Gemini Pro's libraries and its LangChain adapter:

In [ ]:
#!pip install pip install google-generativeai langchain-google-genai

Next you need to acquire an API key, which can be done on the [**Google MakerSuite**](https://makersuite.google.com/). In the top left of the page you should see a "Get API Key" button.

Copy the new API Key and save it to a <font color='olive'>.env</font> file in your project directory.
> GOOGLE_API_KEY=new_api_key_here

Now we can create a script that calls Gemini Pro via LangChain.

In [ ]:
import os

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [ ]:
tweet_prompt = PromptTemplate.from_template(
    "You are a content creator. Write me a tweet about {topic}.")

In [ ]:
tweet_chain = LLMChain(llm=llm, prompt=tweet_prompt, verbose=True)

In [ ]:
topic = "how ai is really cool"

In [ ]:
resp = tweet_chain.run(topic=topic)

In [ ]:
print(resp)

And that's it! You've now integrated Gemini Pro with LangChain! 